In [43]:
# Jupyter notebook needs this or else it will crash
import nest_asyncio
nest_asyncio.apply()

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import P # NEW!!! Import predicates (gt, gte, lt, lte, etc.)
from gremlin_python.process.traversal import Cardinality # NEW!!! Import Cardinality such as list_, set_ and single.
from gremlin_python.driver.protocol import GremlinServerError # Gremlin server error
from gremlin_python.process.traversal import Pop # for Pop.all_ in select(Pop.all_, 'v')

# Instantiate a Gremlin Graph
graph = Graph()

# Connect to the server, instantiate traversal of graph.
g = graph.traversal().withRemote(DriverRemoteConnection('ws://localhost:8182/gremlin','g'))

# Get the vertices of the graph as a list, and print them.

print(g.V().toList())

[v[462936], v[454744], v[123060232], v[123056136], v[487512], v[172096], v[82149544], v[41177184], v[221256], v[82153640], v[163904], v[82120904], v[41156704], v[204872], v[41185440], v[41201808], v[123068424], v[82125000], v[200776], v[446552], v[41164896], v[196680], v[41210000], v[491608], v[147520], v[82145448], v[41181280], v[41177248], v[438360], v[41205904], v[442456], v[213064], v[450648], v[208968], v[41214096], v[41185376], v[483416], v[41173088], v[159808], v[41181344], v[123064328], v[467032], v[479320], v[155712], v[217160], v[168000], v[41168992], v[225352], v[82157736], v[41160800], v[176192], v[143424], v[151616], v[475224], v[82129096], v[471128], v[458840]]


In [44]:
def connected(name1: str, name2: str, time: float) -> bool:
    """
    Given two vertices labelled with <name1> and <name2>, determine whether they were connected at time <time>. 
    Do so by sending a Gremlin query to determine whether there exists an edge between the 
    two vertices such that <time> falls in between their "start_time" and "end_time" parameters.
    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    # Get the vertices associated with the names
    # v1, v2 = g.V().has('name', name1).next(), g.V().has('name', name2).next()

    # Return whether there are edges that:
    #   - connect v1 and v2, 
    #   - labelled 'connection',
    #   - have a start time that is less than or equal to <time>
    #   - either do not have an end time or have an end time that is greater than or equal to <time>
    return g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').and_(
            __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
            __.or_(
                __.hasNot('end'),           # end time doesn't have to exist 
                __.has('end', P.gt(time))  # OR end time must be greater than <time>
            )
        ).count().next() > 0

In [45]:
def set_connection(name1: str, name2: str, time: float, connection: bool) -> None:
    """
    Given two vertices labelled with <name1> and <name2>, create a new connection or terminate their existing connection, based on the value of <bool>. Label with time <time>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    if connection:
        if not connected(name1, name2, time):

            # Add an edge labelled 'connection' with a start time of <time>
            g.V().has('name', name1).as_("a").V().has('name', name2).as_("b").addE('connection').from_("a").to("b").property('start', time).iterate()

    else:
        # For all edges between v1 and v2 labelled 'connection' (there should only be one) that do not have an 'end' property, create an end property of <time>.
        g.V().has('name', name1).bothE('connection').as_('e').bothV().has('name', name2).select('e').hasNot('end').property('end', time).iterate()

In [46]:
def add_type(type: str) -> None:
    """
    Add a vertex to the graph with a 'type' property of <type>.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    g.addV().property('name', type).iterate()

def set_type(name: str, type: str) -> None:
    """
    Connect the vertex labelled with <name> to a vertex labelled <type> with an edge going into the <type> vertex labelled with "type".

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    g.V().has('name', name).as_("a").V().has('name', type).as_("b").addE("type").from_("a").to("b").iterate()

In [47]:
# This will put it in V-E-V-E-V-...-V form as a list per path.
def find_paths(name1: str, name2: str, avoid_type: str, time: float):
    """
    Given two vertices labelled with <name1> and <name2>, return the paths that connect the vertices by edges that were active at <time> as a list.

    Avoid vertices of type avoid_type.

    TODO: Add sphinx documentation if this will be implemented into the actual Python library.
    """

    return g.V().has('name', name1).repeat(
            __.bothE('connection').and_(
                __.has('start', P.lte(time)),   # want start time to be less than or equal to <time>
                __.or_(
                    __.hasNot('end'),           # end time doesn't have to exist 
                    __.has('end', P.gt(time))  # OR end time must be greater than <time>
                )
            ).otherV().not_(__.outE('type').inV().has('name', avoid_type)).simplePath()
        ).until(__.has('name', name2)).path().toList()

In [48]:
# Clear vertices
g.V().drop().iterate()
g.E().drop().iterate()

"""
(Temporary) Naming scheme:
 - COR######: Correlator input
 - ANT######: Antenna
 - DPF######: Dual-Polarization Feed
 - BLN######: (Active) Balun
 - RFT######: RFoF transmitter
 - OPF######: Optical Fiber
 - RFR######: RFoF receiver
 - ADC######: Analog-to-Digital converter
"""

# Set up the types
types = ['COR', 'ANT', 'DPF', 'BLN', 'RFT', 'OPF', 'RFR', 'ADC']
for type in types:
    add_type(type)


# Correlator node name
cor = 'COR000000'

# Add a correlator input node
g.addV().property('name', cor).iterate()
set_type(cor, 'COR')

# number of HIRAX dishes
dishes = 4

# Connections at every 3.
max_time = 12
connections = [(i * 3, bool(i % 2)) for i in range(0, max_time // 3 + 1)]


# Add the components and connect them at different times.
for i in range(1, dishes + 1):

    # The names of the components to refer to
    ant = f'ANT{str(i).zfill(6)}'
    dpf = f'DPF{str(i).zfill(6)}'
    bln = (f'BLN{str(2 * i - 1).zfill(6)}', f'BLN{str(2 * i).zfill(6)}')
    rft = (f'RFT{str(2 * i - 1).zfill(6)}', f'RFT{str(2 * i).zfill(6)}')
    opf = (f'OPF{str(2 * i - 1).zfill(6)}', f'OPF{str(2 * i).zfill(6)}')
    rfr = (f'RFR{str(2 * i - 1).zfill(6)}', f'RFR{str(2 * i).zfill(6)}')
    adc = (f'ADC{str(2 * i - 1).zfill(6)}', f'ADC{str(2 * i).zfill(6)}')

    g.addV().property('name', ant).iterate()
    g.addV().property('name', dpf).iterate()

    set_type(ant, 'ANT')
    set_type(dpf, 'DPF')

    for ind in (0, 1):
        g.addV().property('name', bln[ind]).iterate()
        g.addV().property('name', rft[ind]).iterate()
        g.addV().property('name', opf[ind]).iterate()
        g.addV().property('name', rfr[ind]).iterate()
        g.addV().property('name', adc[ind]).iterate()

        set_type(bln[ind], 'BLN')
        set_type(rft[ind], 'RFT')
        set_type(opf[ind], 'OPF')
        set_type(rfr[ind], 'RFR')
        set_type(adc[ind], 'ADC')

    for (time, connection) in connections:

            set_connection(name1=ant, name2=dpf, time=time, connection=connection)

            for ind in (0, 1):

                # Pairs of names to connect
                pairs = [(ant, dpf), (dpf, bln[ind]), (bln[ind], rft[ind]), (rft[ind], opf[ind]), (opf[ind], rfr[ind]), (rfr[ind], adc[ind]), (adc[ind], cor)]

                for pair in pairs:
                    set_connection(name1=pair[0], name2=pair[1], time=time, connection=connection)

# EXPORT THE GRAPH.
g.io("path_benchmark_small_graph.xml").write().iterate()


[['io', 'path_benchmark_small_graph.xml'], ['write'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]